### Mesh processing for SimuCell3D

This notebook outlines how to generate a geometry for the SimuCell3D.

In [1]:
import vtk

In [2]:
import os
import sys
import numpy as np
from MeshPrep import isolate_filtered_cells
from MeshPrep import mesh_process_clean, string_to_array

ImportError: /local0/fcarrara/mambaforge/envs/fix-meshes/lib/python3.10/site-packages/vtkmodules/../../../././libcurl.so.4: undefined symbol: nghttp2_option_set_no_rfc9113_leading_and_trailing_ws_validation

In [ ]:
sys.path.append('/nas/groups/iber/Users/Federico_Carrara/Statistics_Collection/EpiStats/src/statistics_collection/')
from StatsAnalytics import prepare_df

#### Cleaning the labels

The labels generated from manual curation should be processesed before passing through the geometry generation. Strongly recommended step. Generates meshes which may be used for manual cell patch selection using paraview.

In [ ]:
voxel_resolution = np.array([0.21, 0.21, 0.39])
label_path = 'path/to/labels.tif'
output_folder='path/folder/to/save/processed/labels'

In [ ]:
from VoxelProcessing import full_label_processing

output_folder = 'path/folder/to/save/processed/labels'
cell_info = full_label_processing(labeled_img=label_path, voxel_resolution=voxel_resolution, output_folder=output_folder, smoothing_iterations=10)
label_path = os.path.join(output_folder, "processed_labels.npy")
filtered_cell_list = cell_info[2]

#### Select labels for Simulations

To obtain labels for simulation one may use napari and select cells manually, or use meshes, and paraview. View how to use paraview [here](Tutorials/LabelSelection.md).

You may want to isolate the cells that do not touch the border instead. To isolate these use the following script and use Paraview's extract functionality as outlined.

In [1]:
cell_stats_df = prepare_df(['/outputs/outputs_v2/output_intestine_sample2_b_curated_segmentation_relabel_seq_s_10_e_6_d_8/processed_labels.tif'])

ImportError: attempted relative import with no known parent package

In [ ]:
mesh_path = "path/to/mesh/folder"
isolated_mesh_path = "path/to/mesh/folder/isolated" # This is where the isolated meshes will be saved.

filtered_cell_list

isolate_filtered_cells(mesh_path, isolated_mesh_path, filtered_cell_list)

#### Mesh Cleaning for SimuCell3D

Produce meshes for SimuCell3D simulation framework. Ensure that the path to labels is of cleaned, processed labels. 

In [1]:
#string_to_array("110 127 136 140 141 151 162 170 177 180 188 201 202 223 231 236 243 244 251 257 280 331 96")
label_list = [111, 112, 114, 125, 126, 129, 134, 137, 139, 140, 143, 145, 152, 154, 167, 168, 169, 171, 172, 209, 88, 91] 
output_dir = '/Users/antanas/BC_Project/Experiment_20_condensed' # This is where the processed meshes will be saved.

# Call the mesh_process_clean function
mesh_process_clean(label_path=label_path, output_dir=output_dir, label_list=label_list, voxel_resolution=voxel_resolution, scale_factor=1e-6, make_shell=True)

: 

: 